In [16]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import os
import autogluon
from autogluon.tabular import TabularDataset,TabularPredictor
from sklearn.model_selection import train_test_split
import utils
import geopandas as gpd

In [2]:
current_dir = Path.cwd()
data_dir = current_dir.parent / "data"

In [3]:
gdf_json_saigai = utils.merge_geojson_files(data_dir / "train" / "浸水想定区域" / "10_計画規模")
display(gdf_json_saigai.head())

305個のgeojsonファイルを全CPUコアを使用して並列読み込みします...
結合完了: 合計 7172706 件のフィーチャー


,A31a_101,A31a_102,A31a_103,A31a_104,A31a_105,A31a_106,A31a_107,A31a_108,geometry
0,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.36388 44.26004, 142.36388 44.260..."
1,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.38844 44.25612, 142.38844 44.256..."
2,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.38719 44.25354, 142.38719 44.253..."
3,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.05044 44.83904, 142.05044 44.839..."
4,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.06338 44.8405, 142.06338 44.8405..."


In [3]:
train_df = pd.read_csv(os.path.join("..", "data", "train", "train.csv"))
drop = [
    "reform_place_other",
    "reform_common_area",
    "parking_keiyaku",
    "est_other_distance",
    "building_name_ruby",
    "name_ruby",
    "school_ele_code",
    "school_jun_code",
    "money_hoshou_company",
    "free_rent_duration",
    "free_rent_gen_timing",
    "post2",
    "nl",
    "el",
    "unit_name",
    "money_sonota_str2",
    "basement_floor_count",
    "money_sonota3",
    "land_seigen",
    "snapshot_land_shidou"
]
train_df = train_df.drop(columns=drop)
len(train_df)

C:\Users\km20186\AppData\Local\Temp\ipykernel_23848\4275834969.py:1: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(os.path.join("..", "data", "train", "train.csv"))


363924

In [21]:
map1=utils.generate_year_mapping(1983,'L02_062',104-62,0)
map2=utils.generate_year_mapping(1983,'L02_055',97-55,0)

In [4]:
train_gdf=utils.convert_to_geodataframe(train_df)
#train_gdf=utils.spatial_join(train_gdf,gdf_json_saigai,predicate='within')

In [6]:
print(train_gdf.shape,train_df.shape)

(363924, 130) (363924, 129)


In [5]:
# pandasの表示設定
pd.set_option('display.max_columns', None)  # すべての列を表示
pd.set_option('display.max_rows', None)     # すべての行を表示（オプション）
#pd.set_option('display.width', None)        # 行の折り返しを防ぐ
#pd.set_option('display.max_colwidth', None) # 列の内容を省略しない

In [ ]:
drop_cols=[
    "nl","el",'building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing'
    ]
land_price_cols=[
    "L02_001",#用途区分
    "L02_006",#調査価格
    "L02_024",#地積
    "L02_029",#水道
    "L02_037",#道路区分
    "L02_045",#最寄り駅までの道路距離
    "L02_055",
    "L02_056",
    "L02_057",
    "L02_058",
    "L02_059",
    "L02_060",
    "L02_061",
    "L02_062",
    "L02_063",
    "L02_064",
    "L02_065",
    "L02_066",
    "L02_067",
    "L02_068",
    "L02_069",
    "L02_070",
    "L02_071",
    "L02_072",
    "L02_073",
    "L02_074",
    "L02_075",
    "L02_076",
    "L02_077",
    "L02_078",
    "L02_079",
    "L02_080",
    "L02_081",
    "L02_082",
    "L02_083",
    "L02_084",
    "L02_085",
    "L02_086",
    "L02_087",
    "L02_088",
    "L02_089",
    "L02_090",
    "L02_091",
    "L02_092",
    "L02_093",
    "L02_094",
    "geometry"
    ]

In [6]:
gdf_json = gpd.read_file(os.path.join('..', 'data', 'train', 'S12-24_NumberOfPassengers.geojson'))

In [7]:
rosen=gdf_json.groupby(by="S12_001g").agg(
    total_2023=("S12_057","sum"),
    total_2022=("S12_053","sum"),
    name=("S12_001","first"),
)
rosen=rosen.replace({0:np.nan})
display(rosen.head())

,total_2023,total_2022,name
S12_001g,,,
000001,114.0,NaN,稚内
000002,82.0,NaN,南稚内
000003,NaN,NaN,抜海
000004,NaN,NaN,勇知
000005,NaN,NaN,兜沼


In [8]:
train_gdf=train_gdf.merge(rosen,how='left',left_on="eki_name1",right_on="name")

In [9]:
gdf_json = gpd.read_file(os.path.join('..', 'data', 'train', 'L01-24.geojson'))#[land_price_cols]
gdf_json2 = gpd.read_file(os.path.join('..', 'data', 'train', 'L02-24.geojson'))#[land_price_cols]

# L02_098～L02_134のカラムを削除
drop_cols = [f'L01_{i:03d}' for i in range(105, 147)]
drop_cols.extend([f'L01_{i:03d}' for i in range(10, 27)])
drop_cols.extend([f'L01_{i:03d}' for i in range(62, 98)])
drop_cols.extend([f'L01_{i:03d}' for i in range(51, 57)])
drop_cols.append('L01_103')
print(drop_cols)
gdf_json = gdf_json.drop(columns=[col for col in drop_cols if col in gdf_json.columns], errors='ignore')

# L02については72-97のカラムのみを抽出
keep_cols = [f'L02_{i:03d}' for i in range(90, 98)]
keep_cols.append('geometry')  # geometry列も保持
gdf_json2 = gdf_json2[[col for col in keep_cols if col in gdf_json2.columns]]

['L01_105', 'L01_106', 'L01_107', 'L01_108', 'L01_109', 'L01_110', 'L01_111', 'L01_112', 'L01_113', 'L01_114', 'L01_115', 'L01_116', 'L01_117', 'L01_118', 'L01_119', 'L01_120', 'L01_121', 'L01_122', 'L01_123', 'L01_124', 'L01_125', 'L01_126', 'L01_127', 'L01_128', 'L01_129', 'L01_130', 'L01_131', 'L01_132', 'L01_133', 'L01_134', 'L01_135', 'L01_136', 'L01_137', 'L01_138', 'L01_139', 'L01_140', 'L01_141', 'L01_142', 'L01_143', 'L01_144', 'L01_145', 'L01_146', 'L01_010', 'L01_011', 'L01_012', 'L01_013', 'L01_014', 'L01_015', 'L01_016', 'L01_017', 'L01_018', 'L01_019', 'L01_020', 'L01_021', 'L01_022', 'L01_023', 'L01_024', 'L01_025', 'L01_026', 'L01_062', 'L01_063', 'L01_064', 'L01_065', 'L01_066', 'L01_067', 'L01_068', 'L01_069', 'L01_070', 'L01_071', 'L01_072', 'L01_073', 'L01_074', 'L01_075', 'L01_076', 'L01_077', 'L01_078', 'L01_079', 'L01_080', 'L01_081', 'L01_082', 'L01_083', 'L01_084', 'L01_085', 'L01_086', 'L01_087', 'L01_088', 'L01_089', 'L01_090', 'L01_091', 'L01_092', 'L01_093'

In [10]:
#train_gdf=utils.convert_to_geodataframe(train_df)
train_gdf=utils.nearest_merge(train_gdf,gdf_json,threshold=1000)
train_gdf=utils.nearest_merge(train_gdf,gdf_json2,threshold=1000)
#display(train_gdf.head())


In [12]:
len(train_gdf.columns)

186

In [ ]:
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))
test_gdf=utils.convert_to_geodataframe(test_df)
test_gdf=utils.nearest_merge(test_gdf,gdf_json,threshold=1500)
test_gdf["land_price_x"]=test_gdf.geometry.x
test_gdf["land_price_y"]=test_gdf.geometry.y
test_df = pd.DataFrame(test_gdf.drop(columns=[col for col in test_gdf.columns if 'geometry' in col]))

C:\Users\km20186\AppData\Local\Temp\ipykernel_32892\1463719098.py:1: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [15]:
# distance列がNULLとなっている割合を計算
null_ratio = train_gdf['distance'].isnull().sum() / len(train_gdf)
print(f"distance列のNULL割合: {null_ratio:.2%} ({train_gdf['distance'].isnull().sum()}/{len(train_gdf)})")


distance列のNULL割合: 12.05% (43843/363924)


In [11]:
# GeoDataFrameをDataFrameに変換(geometry列を削除)
train_gdf["land_price_x"]=train_gdf.geometry.x
train_gdf["land_price_y"]=train_gdf.geometry.y
train_df = pd.DataFrame(train_gdf.drop(columns=[col for col in train_gdf.columns if 'geometry' in col]))

In [78]:
len(train_df)

383555

In [12]:
train, test = train_test_split(train_df, test_size=0.2, random_state=42)
train_data: pd.DataFrame = TabularDataset(train)
test_data: pd.DataFrame = TabularDataset(test)
model=TabularPredictor(
    label='money_room',
    problem_type='regression',
    eval_metric='mape',
)
predictor=model.fit(
    train_data=train_data,
    test_data=test_data,
    presets='medium_quality',
    time_limit=720,
)

No path specified. Models will be saved in: "AutogluonModels\ag-20260108_093440"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       14.90 GB / 31.66 GB (47.1%)
Disk Space Avail:   56.47 GB / 237.73 GB (23.8%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 720s
AutoGluon will save models to "c:\Users\km20186\project\signature\my_code\AutogluonModels\ag-20260108_093440"
Train Data Rows:    343376
Train Data Columns: 186
Label Column:       money_room
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    16481.03 MB
	Train Data (Original)  Memory Usage: 175

[1000]	valid_set's l2: 3.04511e+13	valid_set's mean_absolute_percentage_error: -0.164208
[2000]	valid_set's l2: 2.67186e+13	valid_set's mean_absolute_percentage_error: -0.154158
[3000]	valid_set's l2: 2.52332e+13	valid_set's mean_absolute_percentage_error: -0.149227
[4000]	valid_set's l2: 2.42199e+13	valid_set's mean_absolute_percentage_error: -0.145784
[5000]	valid_set's l2: 2.3496e+13	valid_set's mean_absolute_percentage_error: -0.143031
[6000]	valid_set's l2: 2.29969e+13	valid_set's mean_absolute_percentage_error: -0.140818
[7000]	valid_set's l2: 2.25435e+13	valid_set's mean_absolute_percentage_error: -0.138583
[8000]	valid_set's l2: 2.21813e+13	valid_set's mean_absolute_percentage_error: -0.136894
[9000]	valid_set's l2: 2.18653e+13	valid_set's mean_absolute_percentage_error: -0.135166
[10000]	valid_set's l2: 2.15727e+13	valid_set's mean_absolute_percentage_error: -0.133622


	-0.1336	 = Validation score   (-mean_absolute_percentage_error)
	420.04s	 = Training   runtime
	0.95s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 131.14s of the 131.13s of remaining time.
	Fitting with cpus=14, gpus=0, mem=2.4/14.2 GB


[1000]	valid_set's l2: 2.7647e+13	valid_set's mean_absolute_percentage_error: -0.154007


	Ran out of time, early stopping on iteration 1223. Best iteration is:
	[1223]	valid_set's l2: 2.67629e+13	valid_set's mean_absolute_percentage_error: -0.150833
	-0.1508	 = Validation score   (-mean_absolute_percentage_error)
	131.96s	 = Training   runtime
	0.22s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the -1.79s of remaining time.
	Ensemble Weights: {'LightGBMXT': 0.923, 'LightGBM': 0.077}
	-0.1335	 = Validation score   (-mean_absolute_percentage_error)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 723.37s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 2912.3 rows/s (3434 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("c:\Users\km20186\project\signature\my_code\AutogluonModels\ag-20260108_093440")


In [11]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.146985,-0.143271,mean_absolute_percentage_error,15.98495,0.646749,592.879773,15.98495,0.646749,592.879773,1,True,1
1,WeightedEnsemble_L2,-0.146985,-0.143271,mean_absolute_percentage_error,16.06017,0.647752,592.891341,0.07522,0.001003,0.011568,2,True,2


In [15]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.133033,-0.133513,mean_absolute_percentage_error,27.010612,1.179119,552.032809,0.032720,0.000000,0.026354,2,True,3
1,LightGBMXT,-0.133145,-0.133622,mean_absolute_percentage_error,22.606178,0.954127,420.042369,22.606178,0.954127,420.042369,1,True,1
2,LightGBM,-0.149793,-0.150833,mean_absolute_percentage_error,4.371713,0.224992,131.964086,4.371713,0.224992,131.964086,1,True,2


In [66]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.143957,-0.142602,mean_absolute_percentage_error,26.518095,1.179842,624.813503,0.141580,0.001001,0.035886,2,True,3
1,LightGBMXT,-0.145846,-0.143903,mean_absolute_percentage_error,18.199548,0.812299,374.009580,18.199548,0.812299,374.009580,1,True,1
2,LightGBM,-0.153466,-0.153726,mean_absolute_percentage_error,8.176967,0.366542,250.768037,8.176967,0.366542,250.768037,1,True,2


In [47]:
test_data: pd.DataFrame = TabularDataset(test_df)
predictions = predictor.predict(test_data)
#

In [48]:
pred=pd.DataFrame(predictions)
pred.head()
pred.to_csv(os.path.join('..', 'data', 'submit', 'submission.csv'), index_label='id')

In [53]:
len(test.sample(n=1000))

1000

In [13]:
importance_df = predictor.feature_importance(test.sample(n=1000), num_shuffle_sets=1)

Computing feature importance via permutation shuffling for 186 features using 1000 rows with 1 shuffle sets...
	205.5s	= Expected runtime (205.5s per shuffle set)
	117.14s	= Actual runtime (Completed 1 of 1 shuffle sets)


In [ ]:
# カラム名(インデックス)に数字を含む行を除外
importance_df_d = importance_df[importance_df.index.astype(str).str.contains(r'\d', regex=True)]

In [14]:
display(importance_df)

,importance,stddev,p_value,n,p99_high,p99_low
house_area,2.731145e-01,NaN,NaN,1,NaN,NaN
year_built,1.768560e-01,NaN,NaN,1,NaN,NaN
L01_008,1.210749e-01,NaN,NaN,1,NaN,NaN
L02_096,6.935091e-02,NaN,NaN,1,NaN,NaN
L01_001,6.212634e-02,NaN,NaN,1,NaN,NaN
L01_102,5.896445e-02,NaN,NaN,1,NaN,NaN
L01_004,5.103531e-02,NaN,NaN,1,NaN,NaN
snapshot_land_area,3.818507e-02,NaN,NaN,1,NaN,NaN
rosen_name1,3.012146e-02,NaN,NaN,1,NaN,NaN
walk_distance1,1.986258e-02,NaN,NaN,1,NaN,NaN


In [ ]:
drop=["reform_place_other","reform_common_area","parking_keiyaku","est_other_distance",'building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing',]

In [ ]:
"unit_name",
"money_sonota_str2",
"basement_floor_count",
"money_sonota3",
"land_seigen",